In [1]:
# IMPORTS
import numpy as np
import cv2

# VARIABLES
model = './model/res10_300x300_ssd_iter_140000.caffemodel'
proto = './model/face_detect_prototxt.txt'


In [2]:
# detect_face: 
#   Finds all posible bounded areas that could contain a face
#   and draws a box around that area if the confidence in the prediction 
#   is above a certain threshols

def detect_face(image_path, prototxt_path, model_path, min_confidence):
    
    # load our serialized model from disk
    net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)
    
    # load the input image and construct an input blob for the image
    # by resizing to a fixed 300x300 pixels and then normalizing it
    image = None
    if isinstance(image_path, str):
        image = cv2.imread(image_path)
    else: 
        image = image_path
    
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
        (300, 300), (104.0, 177.0, 123.0))
    
    # process image through pretrained net to 
    # obtain detections and their confidence
    net.setInput(blob)
    detections_net = net.forward()
    detections = detections_net[0][0]
    
    # loop over the detections
    for i in range(0, detections_net.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[i][2]
    
        # only display detections above specified confidence
        if confidence > min_confidence:
            # compute the (x, y)-coordinates of the bounding box
            box = detections_net[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
     
            # draw the bounding box of the face along and confidence
            text = "{:.2f}%".format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(image, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
            cv2.putText(image, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    return image

In [6]:
def detect_video_faces(video_path, prototxt_path, model_path, min_confidence): 
    # provide a video processing API layer by passing
    # it through openCV's VideoCapture class
    vidcap = cv2.VideoCapture(video_path)
    
    fps = 30.0
    width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
    height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
    capsize = (width,height)
    
    # Define the codec and create VideoWriter Object
    fourcc = cv2.VideoWriter_fourcc('m', 'p','4','v')
    vout = cv2.VideoWriter()
    vid_success = vout.open('output.mov', fourcc, fps, capsize, True)
    if vid_success:
        success,image = vidcap.read()
        print('processing...')
        while vid_success and success:
            detected = detect_face(image, prototxt_path, model_path, min_confidence)
            vout.write(detected)
            success,image = vidcap.read()
        
        vout.release()
        vidcap.release()
        cv2.destroyAllWindows()
        print('complete')

In [7]:
# IMAGE DETECTION FUNCTION USAGE
# detected_image = detect_face('./testInputs/terry.jpg', proto, model, 0.5)
# cv2.imshow("Output", detected_image)
# cv2.waitKey(0)

# VIDEO DETECTION FUNCTION USAGE
detect_video_faces('./testInputs/input.mov', proto, model, 0.2)

processing...
complete
